In [1]:
import os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [2]:
df_distance = pd.read_csv('input/200407_노선별정류장간거리.csv', encoding='utf-8', dtype={'노선ID':str, 'ISC 정류소 ID':str, '상하행':str})
df_shortest = pd.read_csv('input/정류장간최단거리.csv', dtype={'stop_id':str})
df_shortest = df_shortest.set_index('stop_id')

In [3]:
def get_shortest_dist(f_stop, t_stop):
    return df_shortest.loc[f_stop, t_stop]

In [4]:
df_distance.head()

,0,노선ID,노선명,Visum ID,길이,누적거리,정류장명,순번,순번_정류장ID,ISC 정류소 ID,정류소 번호,정류소 ID,노선명_정류소 번호,상하행
0,1,28001001,9100,595,0.732831,0.000000,숭의역(1번출구),1,1-2802621,2802621,37510.0,163000510.0,9100_37510,0
1,9,28001001,9100,4842,1.462401,0.732831,숭의로터리,2,2-2802809,2802809,37266.0,163000266.0,9100_37266,0
2,25,28001001,9100,1322,2.035351,2.195233,제물포역,3,3-2802635,2802635,37496.0,163000496.0,9100_37496,0
3,43,28001001,9100,1094,1.031002,4.230584,주안사거리,4,4-2802617,2802617,37514.0,163000514.0,9100_37514,0
4,55,28001001,9100,1332,1.457817,5.261585,석바위,5,5-2802728,2802728,37365.0,163000365.0,9100_37365,0


In [5]:
df_shortest.head()

,2800001,2800002,2800003,2800004,2800005,2800006,2800007,2800008,2800009,2800010,...,2807640,2807641,2807642,2807643,2807644,2807645,2807646,2807647,2807648,2807649
stop_id,,,,,,,,,,,,,,,,,,,,,
2800001,0.000000,0.914700,0.378187,0.499773,1.025790,22.227612,22.199614,5.000806,1.474402,1.527610,...,26.859695,16.850793,16.833285,36.764603,34.765652,35.223585,36.384010,36.949184,36.356010,36.441915
2800002,0.914700,0.000000,0.536513,0.414927,0.886510,22.163643,22.131645,5.082774,1.556371,1.609579,...,26.791726,16.783824,16.766316,36.847571,34.846621,35.304554,36.467979,37.030152,36.437978,36.525884
2800003,0.378187,0.536513,0.000000,0.121586,1.403977,22.608799,22.576801,4.622619,1.096215,1.149423,...,27.238882,17.227979,17.209471,36.387416,34.387465,34.843398,36.007823,36.568997,35.976823,36.064728
2800004,0.499773,0.414927,0.121586,0.000000,1.301438,22.577570,22.547572,4.667847,1.141444,1.194652,...,27.206653,17.196751,17.181243,36.432644,34.431694,34.889627,36.052052,36.615225,36.023051,36.109957
2800005,1.025790,0.886510,1.403977,1.301438,0.000000,21.274133,21.247135,5.856716,2.442881,2.496089,...,25.906216,15.897314,15.879806,37.323392,35.321442,35.779375,36.943800,37.505973,36.911799,36.999705


In [22]:
shutil.rmtree('output/distance', ignore_errors=True)
shutil.rmtree('output/shortest', ignore_errors=True)
shutil.rmtree('output/ratio', ignore_errors=True)
os.mkdir('output/distance')
os.mkdir('output/shortest')
os.mkdir('output/ratio')

prv_route_id = None; prv_route_name = None; prv_route_up_and_down = None
prv_stop_id = None; prv_stop_isc_id = None; prv_length = None
df_stop_ids = pd.DataFrame(columns=['stop_id', 'stop_isc_id'])
df_curv = pd.DataFrame(columns=['노선ID', '노선명', '상하행', '노선길이', '굴곡도합계', '셀개수', '굴곡도'])
mat_distance = pd.DataFrame()
curv_sum = 0; cell_num = 0; length_sum = 0

df_dist = df_distance
# df_dist = df_distance[df_distance['노선ID'] == '28511016']
for i, row in tqdm_notebook(df_dist.iterrows(), total=df_dist.shape[0]):
    route_id = row['노선ID']
    route_name = row['노선명']
    route_up_and_down = row['상하행']
    length = row['길이']
    stop_isc_id = row['ISC 정류소 ID']
    stop_id = row['순번_정류장ID']
    
    if prv_route_id != route_id or prv_route_up_and_down != route_up_and_down:
        if len(mat_distance) > 0:
            und = '상행' if prv_route_up_and_down == '0' else '하행'
            mat_distance.to_csv('output/distance/d_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
            mat_shortest.to_csv('output/shortest/s_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
            mat_ratio.to_csv('output/ratio/r_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
            df_curv = df_curv.append({'노선ID': prv_route_id, '노선명': prv_route_name, '상하행': und, '노선길이': length_sum, 
                                      '굴곡도합계': curv_sum, '셀개수': cell_num, '굴곡도': curv_sum / cell_num}, ignore_index=True)
            mat_distance = mat_distance.iloc[0:0]
            mat_shortest = mat_shortest.iloc[0:0]
            mat_ratio = mat_ratio.iloc[0:0]
        
        stops = df_dist[(df_dist['노선ID'] == route_id) & (df_dist['상하행'] == route_up_and_down)]['순번_정류장ID']
        mat_distance = pd.DataFrame(columns=stops, index=stops)
        mat_distance.reset_index(level=0)
        mat_distance = mat_distance.fillna('')
        mat_shortest = pd.DataFrame(columns=stops, index=stops)
        mat_shortest.reset_index(level=0)
        mat_shortest = mat_shortest.fillna('')
        mat_ratio = pd.DataFrame(columns=stops, index=stops)
        mat_ratio.reset_index(level=0)
        mat_ratio = mat_shortest.fillna('')
        
        prv_route_id = None; prv_route_name = None; prv_route_up_and_down = None
        prv_stop_id = None; prv_stop_isc_id = None; prv_length = None
        df_stop_ids = df_stop_ids.iloc[0:0]
        curv_sum = 0; cell_num = 0; length_sum = 0
    
    mat_distance.loc[stop_id, stop_id] = 0
    mat_shortest.loc[stop_id, stop_id] = 0
    mat_ratio.loc[stop_id, stop_id] = 0
    
    if len(df_stop_ids) > 0:
        for _, df_row in df_stop_ids.iterrows():
            if prv_stop_id != None:
                mat_distance.loc[df_row['stop_id'], stop_id] = mat_distance.loc[df_row['stop_id'], prv_stop_id] + prv_length
            else:
                mat_distance.loc[df_row['stop_id'], stop_id] = prv_length
            mat_shortest.loc[df_row['stop_id'], stop_id] = get_shortest_dist(df_row['stop_isc_id'], stop_isc_id)
            if mat_shortest.loc[df_row['stop_id'], stop_id] != 0:
                curv = mat_distance.loc[df_row['stop_id'], stop_id] / mat_shortest.loc[df_row['stop_id'], stop_id]
                curv_sum = curv_sum + curv
                cell_num = cell_num + 1
                mat_ratio.loc[df_row['stop_id'], stop_id] = curv
            else:
                mat_ratio.loc[df_row['stop_id'], stop_id] = 'inf'
            
#             if mat_distance.loc[df_row['stop_id'], stop_id] < mat_shortest.loc[df_row['stop_id'], stop_id]:
#                 print(route_id, route_name, route_up_and_down, prv_stop_id, "->", stop_id)
    
    prv_route_id = route_id
    prv_route_name = route_name
    prv_route_up_and_down = route_up_and_down
    prv_stop_id = stop_id
    prv_stop_isc_id = stop_isc_id
    prv_length = length
    length_sum = length_sum + prv_length
    df_stop_ids = df_stop_ids.append({'stop_id': stop_id, 'stop_isc_id': stop_isc_id}, ignore_index=True)

if len(mat_distance) > 0:
    und = '상행' if prv_route_up_and_down == '0' else '하행'
    mat_distance.to_csv('output/distance/d_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
    mat_shortest.to_csv('output/shortest/s_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
    mat_ratio.to_csv('output/ratio/r_' + prv_route_id + '_' + prv_route_name + '_' + und + '.csv', encoding='utf-8-sig')
    df_curv = df_curv.append({'노선ID': prv_route_id, '노선명': prv_route_name, '상하행': und, '노선길이': length_sum, 
                                      '굴곡도합계': curv_sum, '셀개수': cell_num, '굴곡도': curv_sum / cell_num}, ignore_index=True)
    
df_curv.to_csv('output/curv.csv', encoding='utf-8-sig')